In [1]:
with open("res/infer_results.txt","r",encoding="utf-8") as f:
    res=f.read().splitlines()

filename=[]
data=[]
for item in res:
    filename.append(item.split("\t")[0].split("/")[-1])
    data.append(item.split("\t")[1])

In [2]:
import json
res=[]
for item in data:
    dataitem=json.loads(item)["ocr_info"]
    res.append(dataitem)

In [3]:
with open("train_data/fp/label_name.txt","r",encoding="utf-8") as f:
    label_list=f.read().splitlines()
key_name=[]
for item in label_list:
    first=item.split("_")[0]
    if first not in key_name:
        key_name.append(first)
del key_name[0]

key_eng=["GENDER","AGE","NAME","OCODE","OTYPE","IN","OUT","DAY","PC","PH","JY","OPEN","SK","FH","JS","XX","SKDW",
        "YBLX","HJ","ZC","JC","HY","ZL","SS","WSCL","XY","ZYYP","ZCYF","YBZL","CW","HL","GH","QT","YBTC","TC",
        "XJ","ZH","GRZF","QTZF","ZFY","ZFE"]
        
token2label={}
for a,b in zip(key_name,key_eng):
    token2label[a]=b

label2token={}
for key,value in token2label.items():
    label2token[value]=key

In [4]:
result=[]
for item in res:
    itemjson={}
    for i in item:
        if i["pred"]!="O" and i["pred"] not in itemjson.keys():
            itemjson[i["pred"]]=i["transcription"]
    result.append(itemjson)

In [5]:
from datasets import load_dataset
traindata=load_dataset("csv",data_files="datasets/onlyLayoutXLM.csv")
mark=traindata["train"]
traindata=traindata["train"]
markdata={}
value={}
name="0055f88d65_20.jpg"
lastname=""
for item in mark:
    if item["图名"]!=name:
        markdata[lastname]=value
        value={}
        name=item["图名"]
    value[item["属性名"]]=item["正确值"]
    lastname=item["图名"]

markdata[mark[-1]["图名"]]=value

for key,value in markdata.items():
    if "入院日期" in value.keys():
        value["住院日期"]=value["入院日期"]
        value["入院时间"]=value["入院日期"]
    if "出院日期" in value.keys():
        value["出院时间"]=value["出院日期"]
    if "住院天数" in value.keys():
        value["住院日数"]=value["住院天数"]
        value["共住院"]=value["住院天数"]
    if "（小写）" in value.keys():
        value["￥"]=value["（小写）"]
    if "票据号码" in value.keys():
        value["No"]=value["票据号码"]
        value["NO"]=value["票据号码"]
    if "收款单位" in value.keys():
        value["收款单位（章）"]=value["收款单位"]
        value["收款单位（盖章）"]=value["收款单位"]
    if "收款人" in value.keys():
        value["收款人（章）"]=value["收款人"]
        value["收款人（盖章）"]=value["收款人"]
    if "（小写）" in value.keys():
        value["¥"]=value["（小写）"]

Using custom data configuration default-28db3f41bc5a50eb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to D:\hugging-face\datasets\csv\default-28db3f41bc5a50eb\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
def compare(s1,s2):
    index=[]
    max=0
    data=""
    for l in s2:
        windows=len(list(l))
        for i in range(len(s1)-windows+1):
            num=0
            for j in range(windows):
                if s1[i+j]==l[j]:
                    num+=1
            score=num/windows
            if score>=max:
                index=[]
                index.append(i)
                index.append(i+windows-1)
                max=score
                data=l
        
    if max>=0.66:
        return index,data
    else:
        return -1,-1

In [7]:
l1=["收款单位","收款单位（章）","收款单位（盖章）","收款单位（签章）"]
l2=["收款人","收款人（章）","收款人（盖章）","收款人（签章）"]

In [8]:
import re
f=["：",":",",","，",".","·"]
res=result.copy()
for i in range(len(filename)):
    for j in markdata.keys():
        if j in filename[i] and j.split(".")[0]==filename[i].split(".")[0]:

            sk=""
            date=""
            for key,value in list(result[i].items()):
                first=key.split("_")[0]
                second=key.split("_")[1]
                if first=="SKDW" or first=="SK":
                    sk+=value
                #暂时对小写和大写忽略
                if first=="JS" or first=="HJ":
                    res[i][key]=res[i][key].replace("◆","")
                    res[i][key]=res[i][key].replace("?","")
                    continue

                if key=="XX_VALUE":
                    value=value.replace(" ","")
                    mo=r'[\u4e00-\u9fa5]{1,20}'
                    r=re.findall(mo,value)
                    if len(list(r))>0:
                        value=value.replace(r[0],"")
                        res[i]["XX_VALUE"]=value
                
                if key=="IN_KEY" or key=="IN_VALUE" or key=="OUT_KEY" or key=="OUT_VALUE" and len(value)>18:
                    date+=value
                    
                #如果有值的话忽略
                if second == "VALUE":
                    continue
                else:
                    if first+'_'+"VALUE" in result[i].keys():
                        continue
                    else:
                        index,z=compare(value,markdata[j].keys())
                        if index!=-1 and z!=-1:
                            temp=list(value)
                            if len(value)-1>index[1] and value[index[1]+1] in f:
                                res[i][first+"_"+"KEY"]="".join(temp[index[0]:index[1]+2])
                                res[i][first+"_"+"VALUE"]="".join(temp[index[1]+2:])
                            else:
                                res[i][first+"_"+"KEY"]="".join(temp[index[0]:index[1]+1])
                                res[i][first+"_"+"VALUE"]="".join(temp[index[1]+1:])
            
            index1,z1=compare(sk,l1)
            index2,z2=compare(sk,l2)
            temp=list(sk)
            if index1!=-1 and index2!=-1:
                if len(temp)-1>index1[1] and temp[index1[1]+1] in f:
                    res[i]["SKDW"+"_"+"KEY"]="".join(temp[index1[0]:index1[1]+2])
                    res[i]["SKDW"+"_"+"VALUE"]="".join(temp[index1[1]+2:index2[0]])
                else:
                    res[i]["SKDW"+"_"+"KEY"]="".join(temp[index1[0]:index1[1]+1])
                    res[i]["SKDW"+"_"+"VALUE"]="".join(temp[index1[1]+1:index2[0]])
                
                if len(temp)-1>index2[1] and temp[index2[1]+1] in f :
                    res[i]["SK"+"_"+"KEY"]="".join(temp[index2[0]:index2[1]+2])
                    res[i]["SK"+"_"+"VALUE"]="".join(temp[index2[1]+2:])
                else:
                    res[i]["SK"+"_"+"KEY"]="".join(temp[index2[0]:index2[1]+1])
                    res[i]["SK"+"_"+"VALUE"]="".join(temp[index2[1]+1:])

            if len(date)>18:
                pattern=r'\d{2,4}年\d{1,2}月\d{1,2}日|\d{2,4}\S\d{1,2}\S\d{1,2}|d{2,4}年\d{1,2}月\d{1,2}日\d{1,2}时\d{1,2}分|202\d?|\d{2,4}年\d{1,2}月'
                r=re.findall(pattern,date)
                if len(list(r))>0:
                    res[i]["IN_VALUE"]=r[0]
                    if len(list(r))>1:
                        res[i]["OUT_VALUE"]=r[1]
            break


result=res.copy()
for i in range(len(res)):
    for key,value in list(res[i].items()):
        first=key.split("_")[0]
        second=key.split("_")[1]
        if key=="NAME_VALUE":
            result[i]["NAME_VALUE"]=result[i]["NAME_VALUE"].replace("病历记录","")
        if key=="PH_VALUE":
            result[i]["PH_VALUE"]=result[i]["PH_VALUE"].replace(" ","")
            mo=r'[a-z|A-Z|.]{1,20}'
            r=re.findall(mo,value)
            if len(r)>0:
                result[i]["PH_VALUE"]=result[i]["PH_VALUE"].replace(r[0],"")
        if key=="GENDER_VALUE":
            mo=r'[男|女]'
            r=re.findall(mo,value)
            if len(r)>0:
                result[i]["GENDER_VALUE"]=r[0]
            else:
                result[i]["GENDER_VALUE"]="/"
        if key=="AGE_VALUE":
            mo=r'[\d{1,2}岁]'
            r=re.findall(mo,value)
            if len(r)>0:
                result[i]["GENDER_VALUE"]=r[0]
            else:
                result[i]["GENDER_VALUE"]="/"
        if second=="VALUE" and value=="":
            del result[i][key]

In [9]:
import csv
def insertcsv(csvpath,filename,datajson):
    
    file=open(csvpath,'r',encoding='utf-8')
    keys=[]
    for key in datajson:
        keys.append(key)
    
    datas=csv.reader(file)
    datas=list(datas)
    i=int(len(datas))
    
    for f in range(i):
        if(datas[f][0]==filename):
            for key in keys:
                if datas[f][2]==key:
                    datas[f][3]=datajson[key]
    
    file.close()
    with open(csvpath, 'w', encoding='utf-8',newline='') as csvfile:
        writer  = csv.writer(csvfile)
        for row in datas:
            writer.writerow(row)

In [10]:
def return_res(r,dic,label2token):
    for key,value in r.items():
        first=key.split("_")[0]
        second=key.split("_")[1]
        if second=="VALUE" and label2token[first] in list(dic.keys()):
            dic[label2token[first]]=value
    
    for key,value in r.items():
        first=key.split("_")[0]
        second=key.split("_")[1]
        if second=="KEY" and label2token[first] in list(dic.keys()) and dic[label2token[first]]=="无":
            dic[label2token[first]]="/"

    return dic

In [11]:
mark=[]
r={}
for i in range(len(filename)):
    discharge_summary={
        "性别":"无","年龄":"无","医院名称":"无",
        "组织机构代码":"无","医疗机构类型":"无","入院日期":"无","出院日期":"无","住院天数":"无"
        }
    #购药发票
    medicines_invoice={
        "票据代码":"无","票据号码":"无","校验码":"无",
        "开票日期":"无","收款人":"无","复核人":"无","价税合计（大写）":"无","（小写）":"无"
        }
    #门诊发票
    outpatient_invoice={
        "性别":"无","票据代码":"无","票据号码":"无","校验码":"无","开票日期":"无","收款单位":"无",
        "收款人":"无","复核人":"无","医疗机构类型":"无","医保类型":"无","合计金额（大写）":"无","（小写）":"无",
        "诊查费":"无","检查费":"无","化验费":"无","治疗费":"无","手术费":"无","卫生材料费":"无",
        "西药费":"无","中药饮片":"无","中成药费":"无","一般诊疗费":"无","床位费":"无","护理费":"无",
        "挂号费":"无","其他收费项目":"无","医保统筹基金支付":"无","统筹支付":"无","个人现金支付":"无","个人账户支付":"无",
        "个人自付":"无","其他支付":"无","自付一":"无","自付二":"无"
    }
    #住院发票
    hospitalization_invoice={
        "性别":"无","入院日期":"无","出院日期":"无","住院天数":"无","票据代码":"无","票据号码":"无","校验码":"无","开票日期":"无","收款单位":"无",
        "收款人":"无","复核人":"无","医疗机构类型":"无","医保类型":"无","合计金额（大写）":"无","（小写）":"无",
        "诊查费":"无","检查费":"无","化验费":"无","治疗费":"无","手术费":"无","卫生材料费":"无",
        "西药费":"无","中药饮片":"无","中成药费":"无","一般诊疗费":"无","床位费":"无","护理费":"无",
        "挂号费":"无","其他收费项目":"无","医保统筹基金支付":"无","统筹支付":"无","个人现金支付":"无","个人账户支付":"无",
        "个人自付":"无","其他支付":"无","自付一":"无","自付二":"无"
    }
    for item in traindata:
        if filename[i].split(".")[0]==item["图名"].split(".")[0] and item["图名"] in filename[i] and item["图名"] not in mark and item["材料类型"]=="出院小结":
            mark.append(item["图名"])
            datajson=return_res(result[i],discharge_summary,label2token)
            r[item["图名"]]=datajson
        elif filename[i].split(".")[0]==item["图名"].split(".")[0] and item["图名"] in filename[i] and item["图名"] not in mark and item["材料类型"]=="购药发票":
            mark.append(item["图名"])
            datajson=return_res(result[i],medicines_invoice,label2token)
            r[item["图名"]]=datajson
        elif filename[i].split(".")[0]==item["图名"].split(".")[0] and item["图名"] in filename[i] and item["图名"] not in mark and item["材料类型"]=="门诊发票":
            mark.append(item["图名"])
            datajson=return_res(result[i],outpatient_invoice,label2token)
            r[item["图名"]]=datajson
        elif filename[i].split(".")[0]==item["图名"].split(".")[0] and item["图名"] in filename[i] and item["图名"] not in mark and item["材料类型"]=="住院发票":
            mark.append(item["图名"])
            datajson=return_res(result[i],hospitalization_invoice,label2token)
            r[item["图名"]]=datajson

In [12]:
r

{'0055f88d65_20.jpg': {'性别': '无',
  '入院日期': '无',
  '出院日期': '无',
  '住院天数': '费票据',
  '票据代码': '无',
  '票据号码': '1015036082',
  '校验码': '无',
  '开票日期': '无',
  '收款单位': '医一险',
  '收款人': '荣',
  '复核人': '无',
  '医疗机构类型': '无',
  '医保类型': '无',
  '合计金额（大写）': '陆万式仟减佰拾伍元陆解建分',
  '（小写）': '0228564',
  '诊查费': '/',
  '检查费': '770.00',
  '化验费': '无',
  '治疗费': '10548.50',
  '手术费': '无',
  '卫生材料费': '无',
  '西药费': '1634.73',
  '中药饮片': '无',
  '中成药费': '无',
  '一般诊疗费': '无',
  '床位费': '无',
  '护理费': '无',
  '挂号费': '无',
  '其他收费项目': '无',
  '医保统筹基金支付': '无',
  '统筹支付': '无',
  '个人现金支付': '无',
  '个人账户支付': '0.',
  '个人自付': '2701681',
  '其他支付': '无',
  '自付一': '无',
  '自付二': '无'},
 '0063d2aff7_11.jpg': {'性别': '无',
  '入院日期': '/',
  '出院日期': '/',
  '住院天数': '收费票据',
  '票据代码': '无',
  '票据号码': '无',
  '校验码': '无',
  '开票日期': '无',
  '收款单位': '预',
  '收款人': '32000.00费用累计：59094.68现金收退费：0.00',
  '复核人': '无',
  '医疗机构类型': '无',
  '医保类型': '/',
  '合计金额（大写）': '无',
  '（小写）': '无',
  '诊查费': '/',
  '检查费': '68.00',
  '化验费': '/',
  '治疗费': '83401.00',
  '手术费': '无',
  '卫

In [13]:
for key,value in r.items():
    insertcsv('datasets/onlyLayoutXLM.csv',key,value)